<a href="https://colab.research.google.com/github/ElAzyzNasrallah/My-first-Machine-Learning-model/blob/main/Building_my_first_ML_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Always follow the Machine Learning Pipeline , this step is not the first one 
First we must deal with our data**




It is a simple example of Data Segregation using KFold just to understand it 

In [11]:
print('{} {:61} {}'.format('Round','Training Set','Test set'))
for iteration,data in enumerate(KFdataset,start=1):
  print('{:^9} {} {:^25}'.format(iteration,data[0],str(data[1])))

Round Training Set                                                  Test set
    1     [ 4  5  6  7  8  9 10 11 12 13 14 15]         [0 1 2 3]        
    2     [ 0  1  2  3  8  9 10 11 12 13 14 15]         [4 5 6 7]        
    3     [ 0  1  2  3  4  5  6  7 12 13 14 15]       [ 8  9 10 11]      
    4     [ 0  1  2  3  4  5  6  7  8  9 10 11]       [12 13 14 15]      


#Training and Deploying our MachineLearning algorithm

##Training the model 


In [12]:
import warnings
import pandas
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score

In [35]:
filename='forestfires.csv'
names=['X','Y','month','day','FFMC','DMC','DC','ISI','temp','RH','wind','rain','area']
df=pandas.read_csv(filename,names=names)
df.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12),inplace=True) #All data must be numerical
df.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7),inplace=True) #All data must be numerical

In [36]:
array=df.values
X=array[:,0:12]
Y=array[:,12]

In [37]:
max_error_scoring='max_error'
neg_mean_absolute_error_scoring='neg_mean_absolute_error'
r2_scoring='r2'
neg_mean_squared_error_scoring='neg_mean_squared_error'

In [38]:
models=[]
models.append(('LR',LinearRegression()))
models.append(('Lasso',Lasso()))
models.append(('Ridge',Ridge()))
models.append(('EN',ElasticNet()))
models.append(('KNN',KNeighborsRegressor()))
models.append(('CART',DecisionTreeRegressor()))
models.append(('SVR',SVR()))

In [41]:
#Evaluate models and print results
#results=[]
#names=[]
#for name, model in models:
  #kfold = KFold(n_splits=10, random_state=7)
  #cv_results = cross_val_score(model, X, Y, cv=kfold, scoring=max_error_scoring)
 # cv_results2 = cross_val_score(model, X, Y, cv=kfold, scoring=neg_mean_absolute_error_scoring)
 # cv_results3 = cross_val_score(model, X, Y, cv=kfold, scoring=r2_scoring)
  #cv_results4 = cross_val_score(model, X, Y, cv=kfold, scoring=neg_mean_squared_error_scoring)
 # msg="%s: max error: %f, mean absolute error: %f, r2 error: %f, mean squared error: %f" % (name, cv_results.mean(), -cv_results2.mean(), cv_results3.mean(), -cv_results4.mean())
  #print(msg)

Here we compared the algorithms relying on the metrics scoring we used and we finalized by choosing the **Lasso** Algorithm

In [42]:
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.2, random_state=1, shuffle=True)

In [43]:
lasso_model = Lasso()
lasso_model.fit(X_train, Y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [44]:
predictions=lasso_model.predict(X_test)
print(predictions)

[ 2.94995557 10.31741163  5.11331167  8.82953603  9.37472473  2.25890375
 16.70956165 10.21372406 12.71398315 17.82743566 12.00661147 -0.13366089
  2.05100125  9.61362391 20.35978817  4.46999326 11.4878202   8.22013444
 22.99636305  6.57213204 15.16990813  2.06185639  2.60841973 20.26525161
 17.97317314 15.84725657 11.22294233 18.07415259 19.27700019  6.71476154
 11.69520281  7.95309194 12.58956125  7.84373939 12.95490693  7.86654371
 10.29230298 13.12601655  7.55670958 18.24618339 10.86374712  0.84185952
 20.22670534  8.67143132 17.11750646  9.87835433 -0.41212642 12.75284261
  7.16986389  4.50527181 26.3280149  21.87386519 20.89573415 16.78773238
 21.87773448 16.31145194 12.17760756 21.10444723 19.24150053 13.21816328
  8.7358127   2.11532756 23.43585742  8.45085449 13.32288157  1.14739124
  7.81763591 13.16758569 12.31864714  7.13142564 21.82793694 15.85504258
  7.50956744  9.03027599  9.14037809 10.44621761 19.90381093 22.18118018
 10.29641445 15.17851613  8.17927973 16.67198374  9

##Deploying the Model

In [48]:
import pickle
pickle.dump(lasso_model,open('model.pkl','wb'))

In [49]:
!pip install flask-ngrok

In [55]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
import requests
import numpy as np
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

In [56]:
@app.route('/predict',methods=['POST'])
def home():
    X = int(request.args.get('X', ''))
    Y = int(request.args.get('Y', ''))
    month = int(request.args.get('month', ''))
    day = int(request.args.get('day', ''))
    FFMC = float(request.args.get('FFMC', ''))
    DMC = float(request.args.get('DMC', ''))
    DC = float(request.args.get('DC', ''))
    ISI = float(request.args.get('ISI', ''))
    temp = float(request.args.get('temp', ''))
    RH = float(request.args.get('RH', ''))
    wind = float(request.args.get('wind', ''))
    rain = float(request.args.get('rain', ''))
    prediction = lasso_model.predict([[X, Y, month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain]])
    print('*******************************')
    print(prediction)
    return 'Prediction is ' + str(prediction[0])
      
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cc51-34-125-108-213.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Oct/2021 19:55:00] "POST /predict?X=7&Y=5&month=3&day=5&FFMC=86.2&DMC=26.2&DC=94.3&ISI=5.1&temp=8.2&RH=51&wind=6.7&rain=0.0 HTTP/1.1" 200 -


*******************************
[6.0085497]


127.0.0.1 - - [12/Oct/2021 20:14:04] "POST /predict?X=6&Y=5&month=3&day=1&FFMC=90.1&DMC=39.7&DC=86.6&ISI=6.2&temp=15.2&RH=27&wind=3.1&rain=0.0 HTTP/1.1" 200 -


*******************************
[7.34248477]
